In [1]:
import json

In [2]:
json_file = json.load(open('data_level0.json', encoding="utf8"))
keys = json_file.keys()
print(keys)

dict_keys(['dev', 'test', 'train'])


In [3]:
def extract_data(key: str, json_file: json) -> dict:
    print(len(json_file[key]))

for key in keys:
    print(key)
    extract_data(key, json_file)
    print("\n\n")

dev
1108



test
2610



train
9989





# Pre-processing Rules

- Replace '’' with '''. (Issue: mixture of apostrophe)

In [4]:
# Use Train dataset for testing
for key in ['train']:
    
    # Extract all the dialogue in given array
    # dia0_utt0 -> Dialogue 0, Utterance 0
    dialogue_list = json_file[key]
    #print(dialogue_list.keys())
    break

# Layer 1

- Count the distribution of emotion
- Count the number of sentiments

# Layer 2

- Count the number of utterances and distribution (Histogram)
- Count the number of tokens and distribution (Histogram)